# Trabalho de Análise de Algoritmos - Análise empírica

Propõe-se neste trabalho realizar uma análise algoritmica empírica de um programa considerado "útil". Desta forma, escolheu-se o algoritmo de classificação K-Nearest Neighbors (K-NN).

O K-NN é um algoritmo de aprendizagem de máquina que utiliza uma projeção N-dimensional dos dados. Por meio desta projeção e uma quantidade de dados conhecida (rotulados de forma a saber a quais classes pertence cada dado), ele classifica um novo dado informando a sua classe correta por meio da verificação dos `K` vizinhos mais próximos a ele. É utilizado uma quantidade ímpar de vizinhos verificados a cada novo dado para não haver condição de empate. Ao julgar quais são as classes dos vizinhos mais próximos, o novo dado é classificado de acordo com a classe mais votada.

Um exemplo de aplicação do algoritmo de classificação K-NN é mostrado no gráfico a seguir, no qual um novo exemplo (círculo vermelho de interrogação) é calculado a distância entre os vizinhos mais próximos para calcular o novo exemplo como um **Salmão** ou um **Robalo**.

![alt text](grafico_knn.png "Gráfico do Robalos e Salmões")

## Preparando um ambiente Python 3.x

Antes de apresentarmos os algoritmos, é mostrado a seguir como preparar o ambiente para executar o programa em `Python`, juntamente com as bibliotecas.

#### Primeiramente, vamos criar e preparar o ambiente virtual

Cria-se uma pasta de trabalho e um ambiente virtual python3.x dentro dela:

```
~ $ mkdir trabalho
~ $ cd trabalho/
~/trabalho $ virtualenv -p python3 .
~/trabalho $ source bin/activate
(trabalho) ~/trabalho $ ls
bin  include  lib
```

O diretório estará da seguinte maneira apresentado acima, com as pastas `bin/`, `include/` e `lib/`. Aparecerá `(trabalho)` no começo da linha de comando, pois o *ambiente virtual foi ativado*. Normalmente criamos uma pasta chamada `src/` (source) para colocar os códigos dentro.

Abaixo criamos o diretório e colocamos todo o código em python que será mostrado neste enunciado em apenas um arquivo:

```
(trabalho) ~/trabalho $ mkdir src
(trabalho) ~/trabalho $ cd src/
# mover o código em python completo dentro da pasta ~/trabalho/src
(trabalho) ~/trabalho/src $ ls
knn.py
```

Por fim, para podermos executar o código `knn.py`, precisamos instalar algumas dependência e posteriormente executar o programa:

```
(trabalho) ~/trabalho/src $ pip install numpy sklearn
Collecting numpy
  Using cached https://files.pythonhosted.org/packages/fc/d1/45be1144b03b6b1e24f9a924f23f66b4ad030d834ad31fb9e5581bd328af/numpy-1.16.4-cp37-cp37m-manylinux1_x86_64.whl
Collecting sklearn
Requirement already satisfied: scikit-learn in /home/rhubner/teste/lib/python3.7/site-packages (from sklearn) (0.21.2)
Requirement already satisfied: joblib>=0.11 in /home/rhubner/teste/lib/python3.7/site-packages (from scikit-learn->sklearn) (0.13.2)
Requirement already satisfied: scipy>=0.17.0 in /home/rhubner/teste/lib/python3.7/site-packages (from scikit-learn->sklearn) (1.3.0)
Installing collected packages: numpy, sklearn
Successfully installed numpy-1.16.4 sklearn-0.0

(trabalho) ~/trabalho/src $ python knn.py
Acurácia: 98.33%
```

## Apresentação do algoritmo

O algoritmo K-NN a seguir é utilizado para a identificação de digitos em uma base de dados. Uma base de dados com imagens de vários dígitos de 0 à 9 é utilizado para "treinar" o algoritmo, ou seja, para criar o gráfico 9-dimensional para classificação. Este código implementa uma versão simples do algoritmo de classificação KNN:

In [1]:
# Realização dos imports das bibliotecas
import numpy as np
from sklearn.datasets import load_digits
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split


def class_histogram(kn_neighbors, n_classes):
    '''Faz a contagem das classes dos k vizinhos mais próximos.
    Recebe o vetor com os k mais próximos.'''
    
    # Cria um vetor c zerado com n_classes posições
    c = [0] * n_classes
    # Para cada vizinho entre os k mais próximos
    for i in kn_neighbors:
        # incrementar contagem
        c[i] += 1
    # retorna contagem das classes dos k vizinhos mais próximos
    return c

def knn(X_train, Y_train, x, k):
    '''Faz a classificação do exemplo x baseado nos k mais próximos em X_train.'''
    
    # Calcula o vetor de distâncias entre x e todos os pontos em X_train
    d = euclidean_distances(x.reshape(1, -1), X_train).reshape(-1)
    
    # Ordena o vetor d e retorna os índices da ordenação em relação ao vetor d original. Não mexe no vetor d.
    # Isto ajuda porque é necessário indexar Y_train das posições correspondentes depois da ordenação!
    idx = np.argsort(d)
    # Calcula a contagem das classes dos k vizinhos mais próximos:
    # Y_train[idx][:k] <-- obtém os rótulos dos k vizinhos mais próximos!
    hist = class_histogram(Y_train[idx][:k], len(set(Y_train)))
    # hist apenas é um vetor [c0, c1, c2, ..., c_nclasses] de forma que c0 é a quantidade de vizinhos mais
    # próximos que são da classe 0, c1 é a quantidade de vizinhos mais proximos que são da classe 1, e 
    # assim por diante.
    
    # O knn classifica x como sendo da classe com a maior quantidade de vizinhos mais próximos. Assim,
    # basta retornar a posição da classe que tem a maior quantidade de vizinhos mais próximos.
    #    ex: se np.argmax(hist) == 0, a classe 0 tem a maioria dos vizinhos mais próximos de x.
    return np.argmax(hist)

As funções declaradas anteriormente realizam a contagem dos vizinhos mais próximos (`class_histogram`) e realiza a classificação de um novo elemento baseado no parâmetro passado (`knn`).

O trecho a seguir realiza o carregamento da base de ddos de digitos e separa 70% das amostras para "treinamento" do algoritmo e 30% para testar os vizinhos mais próximos ("teste" do algoritmo).

In [2]:
# Carrega um dataset ``digits'' do sklearn. Esse dataset é composto de 1797 imagens de dígitos manuscritos 
# 8x8 em 16 tons de cinza. Há 64 características por imagem, uma para o valor de cada pixel.
X, Y = load_digits(return_X_y=True)

# Vamos dividir o dataset com 70% no treino e 30% no teste. X_train é o conjunto de treino com os exemplos
# e Y_train são os gabaritos de cada exemplo de X_train. X_test é o conjunto de treino com os exemplos e 
# Y_test são os gabaritos de cada exemplo de X_test se shuffle=True, "embaralha" o dataset antes de fazer
# a separação.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=True)

misses, hits = 0, 0
k = 3

# Para cada exemplo no conjunto de teste
for i, x in enumerate(X_test):
    # Classificar o exemplo. Se o exemplo estiver correto (for igual do gabarito)
    if knn(X_train, Y_train, x, k) == Y_test[i]:
        hits += 1
    else:
        misses += 1

# A acurácia é dada por acertos / (acertos + erros).
print ("Acurácia: %.02f%%" % (hits / float(hits + misses) * 100))

Acurácia: 98.15%


## Análise do algoritmo completo

Com este código exposto, realize a análise algoritmica empírica do problema e compare com uma análise formal assintótica. Leve em consideração o custo de **ordenações** (Em Python é QuickSort) e o **cálculo da distância euclidiana** dentro da função `knn` (verificar a grandeza). Para o restante dos trechos de código deve se analizar o tamanho da entrada e os loops existentes. Considere também que as funções `load_digits` tem complexidade `O(n)` e `train_test_split` é constante.

Vocês deverão **instrumentar os códigos acima** para contabilizar a **quantidade de instruções** principalmente dentro de loops somado/multiplicado à complexidade de cada função. Além disso, realizar a **análise assintótica** formal do problema e, no final, comparar o custo assintótico e empírico.

## Entrega:

O trabalho poderá ser realizado no máximo em duplas. A entrega será realizada até o dia 09 de julho no moodle. Deve ser entregue um pacote compactado por apenas um aluno do grupo contendo o relatório e os códigos desenvolvidos). Este trabalho valerá **1,5 pontos na média**.